In [15]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data.head()


,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


In [16]:
X = train_data[['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',\
    'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']]

y= train_data[['rainfall']] 

X_train, X_test, y_train, y_test  = train_test_split(X,y, random_state=0)

model = LogisticRegression()
model2 = LinearRegression()
model.fit(X_train, y_train)
model2.fit(X_train, y_train)

prediction  = model.predict(X_test)
prediction2 = model2.predict(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))
print('Accuracy of linear regression classifier on test set: {:.2f}'.format(model2.score(X_test, y_test)))

c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of logistic regression classifier on test set: 0.87
Accuracy of linear regression classifier on test set: 0.46


In [21]:
from sklearn.svm import LinearSVC

clf = LinearSVC().fit(X_train, y_train)
print('Accuracy of Linear SVC classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Linear SVC classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))
test_data.fillna(0, inplace=True)     
test = test_data[['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',\
    'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']]
predictionSVC = clf.predict(test)  
result = pd.DataFrame({'id': test_data['id'], 'prediction': predictionSVC})
result.to_csv('resultSVC.csv', index=False)     

Accuracy of Linear SVC classifier on training set: 0.86
Accuracy of Linear SVC classifier on test set: 0.87


c:\Users\Bee\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
test_data.fillna(0, inplace=True)
test = test_data[['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',\
    'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']]
print(test)

prediction = model.predict(test)
print(prediction) 
result = pd.DataFrame({'id': test_data['id'], 'prediction': prediction})
result.to_csv('result.csv', index=False)  

     pressure  maxtemp  temparature  mintemp  dewpoint  humidity  cloud  \
0      1019.5     17.5         15.8     12.7      14.9      96.0   99.0   
1      1016.5     17.5         16.5     15.8      15.1      97.0   99.0   
2      1023.9     11.2         10.4      9.4       8.9      86.0   96.0   
3      1022.9     20.6         17.3     15.2       9.5      75.0   45.0   
4      1022.2     16.1         13.8      6.4       4.3      68.0   49.0   
..        ...      ...          ...      ...       ...       ...    ...   
725    1020.8     18.2         17.6     16.1      13.7      96.0   95.0   
726    1011.7     23.2         18.1     16.0      16.0      78.0   80.0   
727    1022.7     21.0         18.5     17.0      15.5      92.0   96.0   
728    1014.4     21.0         20.0     19.7      19.8      94.0   93.0   
729    1020.9     22.2         18.8     17.0      13.3      79.0   89.0   

     sunshine  winddirection  windspeed  
0         0.0           50.0       24.3  
1         0.0  